In [ ]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Load the data
train_df = pd.read_excel('C:/Users/asus1/Desktop/train.xlsx')
test_df = pd.read_excel('C:/Users/asus1/Desktop/test.xlsx')

# Preprocess data for Y only
X_train = train_df.drop(['Y'], axis=1).values
y_train = train_df[['Y']].values
X_test = test_df.drop(['Y'], axis=1).values
y_test = test_df[['Y']].values

# Normalize data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

# Load the model architecture and weights
model_json_path = "D:/Deep Learning/Best Model/trained_deep_learning_model.json"
model_weights_path = "D:/Deep Learning/Best Model/trained_deep_learning_model.keras"

with open(model_json_path, 'r') as json_file:
    model_json = json_file.read()

# Load the model
best_model = model_from_json(model_json)
best_model.load_weights(model_weights_path)

# Compile the model
best_model.compile(optimizer='nadam', loss='mean_squared_error')

# Initialize the main window
root = tk.Tk()
root.title("Concrete Compressive Strength Prediction")
root.geometry("900x700")

# Create the main frame
frame = tk.Frame(root, padx=3, pady=10)
frame.pack(fill=tk.BOTH, expand=True)

# Function to create labels and entry fields
def create_label_entry(frame, text, row, col):
    label = tk.Label(frame, text=text, font=('Selawik Semibold', 14), anchor='w', width=25)
    label.grid(row=row, column=col, padx=10, pady=7, sticky='w')
    entry = tk.Entry(frame, font=('Selawik Semibold', 14), bg='#FFFFFF', highlightbackground='#000000', highlightthickness=1, bd=0, width=20)
    entry.grid(row=row, column=col+1, padx=10, pady=5)
    return entry

# Title and information labels
label_title = tk.Label(frame, text='GUI Model for Predicting Concrete CS', font=('Selawik Semibold', 18, 'bold', 'underline'), fg='#000000', bg='#FFFFFF')
label_title.grid(row=0, column=0, columnspan=4, pady=10)
label_authors = tk.Label(frame, text='Developers: Mohamed K. Elshaarawy, Abdelrahman K. Hamed & Mostafa M. Alsaadawi', font=('Selawik Semibold', 14, 'bold'), fg='#000000', bg='#FFFFFF')
label_authors.grid(row=1, column=0, columnspan=4, pady=10)

# Definition of Parameters label
label_params = tk.Label(frame, text='Definition of Parameters', font=('Selawik Semibold', 14, 'bold'), fg='#000000', bg='#FFFFFF')
label_params.grid(row=2, column=0, columnspan=2, pady=10)

# Input labels and entry fields
entries = {}
param_labels = [
    'X1: Cement (kg/m^3)',
    'X2: Blast Furnace Slag (kg/m^3)',
    'X3: Fly Ash (kg/m^3)',
    'X4: Water (kg/m^3)',
    'X5: Superplasticizer (kg/m^3)',
    'X6: Coarse Aggregate (kg/m^3)',
    'X7: Fine Aggregate (kg/m^3)',
    'X8: Age (day)'
]
for i, text in enumerate(param_labels):
    entries[text] = create_label_entry(frame, text, i+3, 0)

# Prediction result label
label_output = tk.Label(frame, text='Output:', font=('Selawik Semibold', 16, 'bold'), fg='#000000', bg='#FFFFFF')
label_output.grid(row=11, column=0, pady=10, sticky='w')
label_result = tk.Label(frame, text='Concrete compressive strength (MPa)', font=('Selawik Semibold', 16, 'bold'), fg='#000000', bg='#FFFFFF')
label_result.grid(row=11, column=1, pady=10, sticky='w')

# Output value label
output_value = tk.Label(frame, text="0.0", font=('Selawik Semibold', 16, 'bold'), fg='#C00000', relief="sunken", width=15)
output_value.grid(row=11, column=2, pady=10)

# Function to clear all entries
def clear_entries():
    for entry in entries.values():
        entry.delete(0, tk.END)
        entry.insert(0, "0.0")
    output_value.config(text="0.0")

# Function to predict the result
def predict():
    try:
        input_values = [float(entries[label].get()) for label in param_labels]
        input_data = np.array([input_values])
        input_data_scaled = scaler_X.transform(input_data)  # Scale the input data
        prediction_scaled = best_model.predict(input_data_scaled)
        prediction = scaler_y.inverse_transform(prediction_scaled.reshape(-1, 1))  # Inverse scale the prediction
        output_value.config(text=f"{prediction[0][0]:.4f}")
    except ValueError:
        messagebox.showerror("Error", "Invalid input. Please enter numeric values.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

# Buttons
button_frame = tk.Frame(frame)
button_frame.grid(row=12, column=0, columnspan=4, pady=20)

clear_button = tk.Button(button_frame, text="Clear", font=('Selawik Semibold', 14), command=clear_entries)
clear_button.grid(row=0, column=0, padx=10)
predict_button = tk.Button(button_frame, text="Predict", font=('Selawik Semibold', 14), command=predict)
predict_button.grid(row=0, column=1, padx=10)
exit_button = tk.Button(button_frame, text="Exit", font=('Selawik Semibold', 14), command=root.quit)
exit_button.grid(row=0, column=2, padx=10)

# Initial values for entries
clear_entries()

# Run the main loop
root.mainloop()
